In [1]:
from datasets import load_dataset
dataset = load_dataset('csv',data_files='KOBIS_202509.csv')
dataset

c:\Users\C118\miniconda3\envs\llmEnv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['순위', '영화명', '개봉일', '매출액 ', '매출액 점유율', '누적매출액 ', '관객수 ', '누적관객수 ', '스크린수 ', '상영횟수 ', '대표국적 ', '국적 ', '배급사 '],
        num_rows: 362
    })
})

In [2]:
# 텍스트가 아닌 테이블 형식의 데이터로 테스트
import pandas as pd
df = pd.DataFrame(data=dataset['train'])
df.columns = df.columns.str.strip()
print(df.columns)

Index(['순위', '영화명', '개봉일', '매출액', '매출액 점유율', '누적매출액', '관객수', '누적관객수', '스크린수',
       '상영횟수', '대표국적', '국적', '배급사'],
      dtype='object')


In [3]:
df["text"] = df.apply(
    lambda row: f"영화 {row['영화명']}는 {row['개봉일']}에 개봉한 {row['국적']} {row['배급사']} 에서 만든 영화 , 총 매출은 {row['누적매출액']}원, 관객수는 {row['누적관객수']}명이다.",
    axis=1
)

In [4]:
df.head()

,순위,영화명,개봉일,매출액,매출액 점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,배급사,text
0,1,극장판 귀멸의 칼날: 무한성편,2025-08-22,15434624000,43.40%,4.948354e+10,1409431,4563381,1761,72728,일본,일본,(주)씨제이이엔엠,영화 극장판 귀멸의 칼날: 무한성편는 2025-08-22에 개봉한 일본 (주)씨제이...
1,2,얼굴,2025-09-11,4503173060,12.70%,4.503173e+09,433547,433547,985,22063,한국,한국,메가박스중앙(주) 플러스엠 엔터테인먼트,영화 얼굴는 2025-09-11에 개봉한 한국 메가박스중앙(주) 플러스엠 엔터테인먼...
2,3,컨저링: 마지막 의식,2025-09-03,4036148990,11.30%,4.039686e+09,383858,384203,918,23653,미국,미국,워너브러더스 코리아(주),영화 컨저링: 마지막 의식는 2025-09-03에 개봉한 미국 워너브러더스 코리아(...
3,4,살인자 리포트,2025-09-05,3191538400,9.00%,3.199620e+09,319058,319956,841,24212,한국,한국,"소니픽쳐스엔터테인먼트코리아주식회사극장배급지점,주식회사 에이투지엔터테인먼트",영화 살인자 리포트는 2025-09-05에 개봉한 한국 소니픽쳐스엔터테인먼트코리아주...
4,5,F1 더 무비,2025-06-25,2706241310,7.60%,5.384700e+10,302656,5063856,730,20435,미국,미국,워너브러더스 코리아(주),영화 F1 더 무비는 2025-06-25에 개봉한 미국 워너브러더스 코리아(주) 에...


In [5]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [6]:
from pinecone import  Pinecone,ServerlessSpec
pine = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

In [7]:
index_name='kobis'

In [8]:
pine.delete_index(index_name)

In [9]:

pine.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(cloud='aws',region='us-east-1')
)

{
    "name": "kobis",
    "metric": "cosine",
    "host": "kobis-dksb1qh.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [10]:

index = pine.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [11]:
# pip install sentence_transformers
from sentence_transformers import SentenceTransformer

# 올바른 모델 호출
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = model.encode(df["text"].tolist())

c:\Users\C118\miniconda3\envs\llmEnv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\C118\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [12]:
from uuid import uuid4
ids = [str(uuid4()) for _ in range(len(df))]


In [13]:
metas = df['text'].apply(lambda x: {"full-text": x}).tolist()


In [14]:
index.upsert(
    vectors=zip(ids,embeddings,metas),
    namespace='kobis-ns1',
)

{'upserted_count': 362}

In [15]:
question=['F1 더 무비 관객수 ']

In [16]:
# 질문을 임베딩하여 벡터로 변환합니다.
embedding = model.encode(question)   # numpy 배열 리턴
print(len(embedding[0]))
print(type(embedding[0]))
print(embedding.shape)


384
<class 'numpy.ndarray'>
(1, 384)


In [17]:
response = index.query(
namespace='kobis-ns1',
vector=embedding[0].tolist(),    # 조회할 값
top_k=3,
include_values=False,
include_metadata=True
)

In [18]:
print(type(response))


<class 'pinecone.core.openapi.db_data.model.query_response.QueryResponse'>


In [19]:
response

{'matches': [{'id': '605980b0-7bed-4a5a-8e15-bcc06db121aa',
              'metadata': {'full-text': '영화 F1 더 무비는 2025-06-25에 개봉한 미국 워너브러더스 '
                                        '코리아(주) 에서 만든 영화 , 총 매출은 '
                                        '53847001410.0원, 관객수는 5063856명이다.'},
              'score': 0.728853226,
              'values': []},
             {'id': 'c7832f09-d658-449e-a7c7-69660bb059d7',
              'metadata': {'full-text': '영화 조선명탐정: 흡혈괴마의 비밀는 2018-02-08에 개봉한 '
                                        '한국 (주)쇼박스 에서 만든 영화 , 총 매출은 '
                                        '19823389764.0원, 관객수는 2444236명이다.'},
              'score': 0.522564888,
              'values': []},
             {'id': '9305e85d-7b14-4922-8d93-3e5a8648d6b9',
              'metadata': {'full-text': '영화 미드웨이는 2019-12-31에 개봉한 미국 (주)누리픽쳐스 '
                                        '에서 만든 영화 , 총 매출은 8247347929.0원, 관객수는 '
                                        '960606명이다.'},
              'score': 0

In [20]:
context = response.matches[0].metadata['full-text']
context

'영화 F1 더 무비는 2025-06-25에 개봉한 미국 워너브러더스 코리아(주) 에서 만든 영화 , 총 매출은 53847001410.0원, 관객수는 5063856명이다.'

In [21]:
from langchain_openai import  ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model='gpt-4.1-mini')

In [22]:
chat_template = ChatPromptTemplate.from_messages([
    ('system','''당신은 영화 통계에 대해 답변합니다. 
                질문에 대해 {context} 활용하여 잘 설명해주세요.'''),
    ('user','''{question} 에 대하여 알려주세요.''')          
])

In [23]:
prompt = chat_template.format_messages(question=question,context=context)
response = llm.invoke(prompt)
print(response.content)

영화 'F1 더 무비'의 관객수는 총 5,063,856명입니다. 이 영화는 2025년 6월 25일에 미국 워너브러더스 코리아(주)에서 제작 및 배급하였으며, 많은 관객들의 사랑을 받아 높은 관객수를 기록했습니다.


In [24]:
# RAG 없이
from openai import OpenAI
client = OpenAI()
question = 'F1 더 무비 2025년 9월 한국 관객수 '
test = client.chat.completions.create(
    model = 'gpt-4.1-mini',
    messages = [
        {"role":"system","content":'''당신은 백과사전 처러 여러가지 지식을 갖고 있습니다. 
                질문에 대해 잘 설명해주세요.'''},
        {"role":"user","content":f'''{question} 에 대하여 지식을 알려주세요.'''}        
    ],
    temperature=0.1
)
print(test.choices[0].message.content)

현재 2024년 6월 기준으로, 2025년 9월에 개봉 예정인 영화 'F1 더 무비'의 한국 관객수에 대한 구체적인 정보는 아직 공개되지 않았습니다. 영화의 관객수는 개봉 후 관객 반응, 마케팅, 경쟁작 등 여러 요인에 따라 달라지기 때문에 사전에 정확한 예측이 어렵습니다.

'F1 더 무비'는 포뮬러 1(Formula 1) 경주를 다룬 다큐멘터리 또는 영화로, 전 세계적으로 F1 팬들에게 큰 관심을 받고 있습니다. 한국에서도 F1 인기가 점차 높아지고 있어, 개봉 시 상당한 관객수를 기록할 가능성이 있습니다.

추후 개봉 시점에 가까워지면 영화 배급사나 영화진흥위원회, 박스오피스 사이트 등을 통해 정확한 관객수 통계가 공개될 것입니다.
